# 1 邻接矩阵和输入

首先构建邻接矩阵

In [1]:
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse
import os
import random
import time
import torch
import torch.optim as optim
import lib.toy_data as toy_data
from lib.toy_data import generate_slope
import lib.utils as utils
from lib.utils import standard_normal_logprob, set_random_seed, standard_uniform_logprob, x2z
from lib.utils import count_nfe, count_total_time
from lib.utils import build_model_tabular, evaluation
from lib.visualize_flow import visualize_transform, standard_fig_save
import lib.layers.odefunc as odefunc
from lib.layers.container import EpsGenerator, MyModel, GNN
from args import add_args
import numpy as np
# from tensorboardX import SummaryWriter
from torch.utils.tensorboard import SummaryWriter
from lib.dataloader import load_features_labels, MyDataSet, load_loc_data
from torch.utils.data import DataLoader
import torch.nn.functional as F

set_random_seed(1024)
device = torch.device('cuda:' + str(0) if torch.cuda.is_available() else 'cpu')

E:\ProgramData\Anaconda3\envs\ManifoldFlow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\ProgramData\Anaconda3\envs\ManifoldFlow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\ProgramData\Anaconda3\envs\ManifoldFlow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\ProgramData\Anaconda3\envs\M

In [2]:
# 从z计算pair wise的距离，并生成邻接矩阵
def normed_A(z):
    # z 是N*3的原始坐标，varrho是限制，生成可以直接使用的A和保留比例
    Dis = torch.cdist(z, z, p=2).float()  # distance matrix
    m = Dis.mean()

    # 不带权A
    A = torch.eye(Dis.size(0), dtype=torch.float32)
    # A[Dis < self.varrho] = 1
    A[Dis < m] = 1
    remain_persent = (A.sum() / (A.shape[0] * A.shape[1])).item()
    print("Sparsity of A(0 is empty):", remain_persent * 100, "%")
    print("Cut persent:", 100 - remain_persent * 100, "%")
    # 带权A
    # A = torch.where(Dis < m, Dis, torch.eye(Dis.size(0), dtype=torch.float32)).float()
    # degree matrix
    degree = A.sum(1)
    D = torch.diag(torch.pow(degree, -0.5))
    normed_A = D.mm(A).mm(D)
    return normed_A

In [3]:
data = "DDH_right"
seq_len = 3
pre_len = 1
batch_size = 5
feature_data_path = r"D:\projects\SF\toy_example\data\{}.csv".format(data)

# 坐标数据
loc_data = np.loadtxt(fname=feature_data_path, delimiter=",", skiprows=1)[:, 0:3]
loc_data = torch.tensor(loc_data)
# normalize
for i in range(3):
    _min = loc_data[:, i].min()
    _max = loc_data[:, i].max()
    loc_data[:, i] = (loc_data[:, i] - _min)/(_max - _min)

N = loc_data.shape[0]

# 特征loader
dataset = MyDataSet(feature_data_path, seq_len, pre_len, device)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
feature_iter = iter(train_loader)
num_nodes = dataset.num_nodes

Shape of D:\projects\SF\toy_example\data\DDH_right.csv: (21, 2164) 


In [4]:
adj = normed_A(loc_data)

Sparsity of A(0 is empty): 49.33028519153595 %
Cut persent: 50.66971480846405 %


In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# 图注意力层的定义
class GraphAttentionLayer(nn.Module):
    def __init__(self, in_c, out_c):
        super(GraphAttentionLayer, self).__init__()
        self.in_c = in_c
        self.out_c = out_c

        self.F = F.softmax

        self.W = nn.Linear(in_c, out_c, bias=False)  # y = W * x
        self.b = nn.Parameter(torch.Tensor(out_c))

        nn.init.normal_(self.W.weight)
        nn.init.normal_(self.b)

    def forward(self, inputs, graph):
        """
        :param inputs: input features, [B, N, C].
        :param graph: graph structure, [N, N].
        :return:
            output features, [B, N, D].
        """

        # eq.1 计算注意力系数e_ij
        h = self.W(inputs)  # [B, N, D] = [B, N, C] * [C, D]
        outputs = torch.bmm(h, h.transpose(1, 2)) * graph.unsqueeze(0)  # [B, N, N]      x(i)^T * x(j)
        outputs.data.masked_fill_(torch.eq(outputs, 0), -float(1e16))   # x(i)|| x(j)

        # 注意力系数归一化
        attention = self.F(outputs, dim=2)   # [B, N, N]
        return torch.bmm(attention, h) + self.b  # [B, N, N] * [B, N, D]


class GATSubNet(nn.Module):
    def __init__(self, in_c, hid_c, out_c, n_heads):
        super(GATSubNet, self).__init__()

        self.attention_module = nn.ModuleList([GraphAttentionLayer(in_c, hid_c) for _ in range(n_heads)])
        self.out_att = GraphAttentionLayer(hid_c * n_heads, out_c)

        self.act = nn.LeakyReLU()

    def forward(self, inputs, graph):
        """
        :param inputs: [B, N, C]
        :param graph: [N, N]
        :return:
        """
        # 将不同的head拼接到一起
        outputs = torch.cat([attn(inputs, graph) for attn in self.attention_module], dim=-1)  # [B, N, hid_c * h_head]
        outputs = self.act(outputs)
        # eq.6 聚合，原文是求平均，这里是一个额外的attention层
        outputs = self.out_att(outputs, graph)

        return self.act(outputs)


# GAT 网络的定义
class GATNet(nn.Module):
    def __init__(self, in_c, hid_c, out_c, n_heads):
        super(GATNet, self).__init__()
        self.subnet = GATSubNet(in_c, hid_c, out_c, n_heads)

    def forward(self, data, graph, device):
        graph = graph.to(device)  # [N, N]
        flow = data.to(device)  # [B, N, T, C]

        B, N = flow.size(0), flow.size(1)
        flow = flow.view(B, N, -1)  # [B, N, T * C]

        # prediction = self.subnet(flow, graph).unsqueeze(2)  # [B, N, 1, C]
        prediction = self.subnet(flow, graph)  # [B, N, C]
        return prediction


In [26]:
gat = GATNet(in_c=seq_len, hid_c=3, out_c=pre_len, n_heads=2).to(device)

loss = torch.nn.MSELoss(reduce=None, size_average=None)
optimizer = optim.Adam(gat.parameters(), lr=0.001, weight_decay=0.9)

In [17]:
for itr in range(5):
    optimizer.zero_grad()
    torch.cuda.empty_cache()
    try:
        x, y = feature_iter.next()
    except StopIteration:
        feature_iter = iter(train_loader)
        x, y = feature_iter.next()
    # x = [B, N, seq_len], y = [B, N, pre_len]
    pre = gat(x, adj, device)
    l = loss(input=pre, target=y)

    print(l.item())

    l.backward()
    optimizer.step()

6.3694071769714355
6.177985668182373
5.815783977508545
5.885026931762695
5.625741481781006


In [18]:
x, y = feature_iter.next()

In [19]:
pre = gat(x, adj, device)

In [24]:
adj.unsqueeze(0).shape

torch.Size([1, 2164, 2164])

In [20]:
pre

tensor([[[2.8491],
         [2.8503],
         [2.8510],
         ...,
         [2.8544],
         [2.8541],
         [2.8509]],

        [[3.0372],
         [3.0553],
         [3.0650],
         ...,
         [2.9490],
         [2.9405],
         [2.9727]],

        [[2.6847],
         [2.6657],
         [2.6554],
         ...,
         [2.7433],
         [2.7463],
         [2.7536]]], device='cuda:0', grad_fn=<LeakyReluBackward0>)